<p style="font-size: 45px; text-align: center;"><b>Machine Learning in Finance II - final project</b></p>
<p style="font-size: 35px; text-align: center;"><b>Forecasting delays in delivery time - Brazilian E-commerce</b></p>
<p style="font-size: 20px; text-align: center;"><b>Summary - choosing the best model</b></p>

Author: Jakub Pyszniak

Summary Notebook

We utilise a similar custom RandomSearchCV set-up for cross-validation with some necessary modifications for each boosting model (especially for CatBoost due to slow performance) This ensures greater comparability of results however possibly ignores the full potential of indvidual tuning methods for each model

Quick ranking:

1) XGBoost
2) CatBoost
3) LightGBM